In [12]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
solve_season = '2024-25'
solve_gameweek = 15
load_projections_from_file = False

In [14]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [15]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [16]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW15 Challenge: Big Spender - Players costing over £9m earn double points

In [17]:
double_points_cost = 9
projections_data.loc[projections_data['Cost'] > double_points_cost, 'Predicted_Points'] *= 2

In [18]:
# Set liverpool and everton players to 0 points on the back of the game cancellation
projections_data.loc[projections_data['Team'].isin(['Liverpool', 'Everton']), 'Predicted_Points'] = 0

### Optimisation

In [19]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 11
model += plp.lpSum(lineup) == 11

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# Between 3-5 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 3
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 5

# Between 2-5 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 2
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 5

# Between 1-3 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) <= 3

# Budget constraint TBC
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) <= 100

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")
        
solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Martinez - Aston Villa - Cost: 5.0m - Predicted Points: 4.3

Defender:
  Cash - Aston Villa - Cost: 4.4m - Predicted Points: 4.68
  Digne - Aston Villa - Cost: 4.7m - Predicted Points: 4.28
  Konsa - Aston Villa - Cost: 4.4m - Predicted Points: 4.08

Midfielder:
  Saka - Arsenal - Cost: 10.5m - Predicted Points: 12.28
  Palmer - Chelsea - Cost: 11.0m - Predicted Points: 14.0
  De Bruyne - Man City - Cost: 9.4m - Predicted Points: 5.68
  Son - Spurs - Cost: 9.8m - Predicted Points: 10.52

Forward:
  Watkins - Aston Villa - Cost: 9.0m - Predicted Points: 6.36
  Haaland (C) - Man City - Cost: 15.0m - Predicted Points: 28.08
  Isak - Newcastle - Cost: 8.5m - Predicted Points: 5.82

Total Predicted Points: 100.08
Total Cost: 91.7m


In [20]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

projections_data.to_csv('gameweek_projections.csv', index=False)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
32,47,Martinez,Aston Villa,10,Goalkeeper,5.0,4.30,90,Southampton (H),0.0,0.0,0.0,0.000,0.0
10,15,Raya,Arsenal,200,Goalkeeper,5.5,4.02,90,Fulham (A),0.0,0.0,0.0,0.444,0.0
56,91,Flekken,Brentford,152,Goalkeeper,4.5,3.99,90,Newcastle (H),0.0,0.0,0.0,0.200,0.0
223,383,Onana,Man Utd,38,Goalkeeper,5.2,3.91,90,Nott'm Forest (H),0.0,0.0,0.0,0.400,0.0
329,568,Muric,Ipswich,286,Goalkeeper,4.4,3.87,90,Bournemouth (H),0.0,0.0,0.0,0.200,0.0
282,488,Forster,Spurs,241,Goalkeeper,4.3,3.69,90,Chelsea (H),0.0,0.0,0.0,0.143,0.0
82,146,Verbruggen,Brighton,152,Goalkeeper,4.5,3.62,90,Leicester (A),0.0,0.0,0.0,0.364,0.0
302,521,Fabianski,West Ham,172,Goalkeeper,4.1,3.62,79,Wolves (H),0.0,0.0,0.0,0.308,0.0
135,248,Leno,Fulham,80,Goalkeeper,5.0,3.58,90,Arsenal (H),0.0,0.0,0.0,0.167,0.0
85,152,Arrizabalaga,Bournemouth,200,Goalkeeper,4.5,3.49,90,Ipswich (A),0.0,0.0,0.0,0.308,0.0


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
24,32,Cash,Aston Villa,172,Defender,4.4,4.68,82,Southampton (H),0.152,0.104,0.240,0.426,0.177
26,36,Digne,Aston Villa,73,Defender,4.7,4.28,70,Southampton (H),0.096,0.200,0.277,0.443,0.107
30,44,Konsa,Aston Villa,241,Defender,4.4,4.08,90,Southampton (H),0.077,0.030,0.105,0.454,0.088
37,52,Pau,Aston Villa,200,Defender,4.4,3.96,90,Southampton (H),0.111,0.044,0.150,0.454,0.123
216,369,Dalot,Man Utd,173,Defender,5.2,3.89,90,Nott'm Forest (H),0.088,0.121,0.198,0.400,0.099
340,593,De Ligt,Man Utd,152,Defender,5.0,3.85,90,Nott'm Forest (H),0.078,0.042,0.116,0.400,0.087
70,122,Estupiñan,Brighton,62,Defender,5.1,3.75,90,Leicester (A),0.098,0.178,0.258,0.364,0.109
301,520,Emerson,West Ham,106,Defender,4.4,3.75,80,Wolves (H),0.014,0.019,0.033,0.292,0.020
287,495,Pedro Porro,Spurs,200,Defender,5.6,3.71,90,Chelsea (H),0.141,0.124,0.247,0.143,0.161
54,88,Collins,Brentford,104,Defender,4.5,3.70,90,Newcastle (H),0.067,0.038,0.102,0.200,0.077


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
101,182,Palmer,Chelsea,241,Midfielder,11.0,14.00,89,Spurs (A),0.434,0.255,0.578,0.222,0.618
12,17,Saka,Arsenal,241,Midfielder,10.5,12.28,86,Fulham (A),0.327,0.300,0.529,0.444,0.420
290,503,Son,Spurs,114,Midfielder,9.8,10.52,67,Chelsea (H),0.327,0.200,0.462,0.143,0.427
197,345,De Bruyne,Man City,21,Midfielder,9.4,5.68,40,Crystal Palace (A),0.042,0.036,0.077,0.364,0.054
214,366,B.Fernandes,Man Utd,173,Midfielder,8.5,5.45,90,Nott'm Forest (H),0.246,0.246,0.431,0.400,0.299
107,199,Eze,Crystal Palace,241,Midfielder,6.6,5.15,87,Man City (H),0.191,0.119,0.288,0.167,0.227
61,99,Mbeumo,Brentford,38,Midfielder,7.7,4.97,90,Newcastle (H),0.344,0.220,0.488,0.200,0.435
297,514,Bowen,West Ham,241,Midfielder,7.4,4.94,90,Wolves (H),0.356,0.261,0.524,0.308,0.456
49,78,Semenyo,Bournemouth,81,Midfielder,5.7,4.86,90,Ipswich (A),0.297,0.161,0.411,0.308,0.364
285,492,Kulusevski,Spurs,206,Midfielder,6.3,4.77,84,Chelsea (H),0.000,0.000,0.000,0.000,0.000


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
203,351,Haaland,Man City,161,Forward,15.0,14.04,90,Crystal Palace (A),0.507,0.152,0.582,0.364,0.756
40,58,Watkins,Aston Villa,241,Forward,9.0,6.36,82,Southampton (H),0.469,0.188,0.569,0.432,0.713
235,401,Isak,Newcastle,206,Forward,8.5,5.82,90,Brentford (A),0.374,0.127,0.454,0.250,0.512
313,541,Cunha,Wolves,30,Forward,7.1,5.43,90,West Ham (A),0.308,0.212,0.455,0.182,0.384
66,110,Wissa,Brentford,50,Forward,6.2,5.27,83,Newcastle (H),0.238,0.122,0.331,0.192,0.291
260,447,Wood,Nott'm Forest,154,Forward,6.5,5.17,80,Man Utd (A),0.219,0.080,0.282,0.198,0.266
112,207,Mateta,Crystal Palace,73,Forward,7.2,5.16,90,Man City (H),0.237,0.089,0.305,0.167,0.288
99,180,N.Jackson,Chelsea,189,Forward,8.2,5.06,83,Spurs (A),0.333,0.124,0.416,0.217,0.441
73,129,João Pedro,Brighton,30,Forward,5.9,5.03,76,Leicester (A),0.180,0.104,0.265,0.359,0.218
51,82,Solanke,Spurs,241,Forward,7.5,5.01,90,Chelsea (H),0.340,0.122,0.421,0.143,0.430
